<a href="https://colab.research.google.com/github/soheilpaper/twitter-personality-classification1/blob/master/Automate_robat/Groq_autogen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install autogen-agentchat~=0.2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 382.3/382.3 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.2/314.2 kB 22.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 34.6 MB/s eta 0:00:00


In [ ]:
import os
from autogen import AssistantAgent, UserProxyAgent

llm_config = { "config_list": [{ "model": "gpt-4", "api_key": os.environ.get("OPENAI_API_KEY") }] }
assistant = AssistantAgent("assistant", llm_config=llm_config)
user_proxy = UserProxyAgent("user_proxy", code_execution_config=False)

# Start the chat
user_proxy.initiate_chat(
    assistant,
    message="Tell me a joke about NVDA and TESLA stock prices.",
)

In [ ]:
import os
from autogen import AssistantAgent, UserProxyAgent

# Configure Groq API
llm_config = {
    "config_list": [
        {
            "model": "llama3-8b-8192",  # Example model supported by Groq
            "api_key": "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", #os.environ.get("GROQ_API_KEY"),
            "base_url": "https://api.groq.com/openai/v1"
        }
    ]
}

# Initialize agents
assistant = AssistantAgent("assistant", llm_config=llm_config)
user_proxy = UserProxyAgent("user_proxy", code_execution_config=False)

# Start the chat
user_proxy.initiate_chat(
    assistant,
    message="Tell me a joke about NVDA and TESLA stock prices."
)

/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


user_proxy (to assistant):

Tell me a joke about NVDA and TESLA stock prices.

--------------------------------------------------------------------------------
[autogen.oai.client: 12-25 09:08:18] {351} WARNING - Model llama3-8b-8192 is not found. The cost will be 0. In your config_list, add field {"price" : [prompt_price_per_1k, completion_token_price_per_1k]} for customized pricing.


assistant (to user_proxy):

Sounds like a challenging task! Let's gather some information first.

To make a joke about NVDA and TESLA stock prices, I'd like to know the current stock prices of both companies. I'll print the current stock prices and then use them to craft a joke.

Please execute the following Python code:
```python
# filename: get_stock_prices.py
import requests

nvidia_response = requests.get('https://www alpha vantage.co/query?function=GLOBAL_QUOTE&symbol=NVDA&apikey=YOUR_API_KEY')
tesla_response = requests.get('https://www alpha vantage.co/query?function=GLOBAL_QUOTE&symbol=TSLA&apikey=YOUR_API_KEY')

print(nvidia_response.json()['Global Quote']['05. price'])
print(tesla_response.json()['Global Quote']['05. price'])
```
Replace `YOUR_API_KEY` with your actual API key from Alpha Vantage or a similar service.

After executing the code, you should see the current stock prices of NVDA and TESLA printed. Now, I can use these prices to craft a joke for you!

Please provide

assistant (to user_proxy):

Let me execute the code to get the current stock prices of NVDA and TESLA. Please provide the output of the code.

**Please note:** You'll need to replace `YOUR_API_KEY` with your actual Alpha Vantage API key. If you don't have one, you can sign up for a free account.

After you provide the output, I'll craft a joke using the stock prices and share it with you!

TERMINATE

--------------------------------------------------------------------------------
